## Objectifs du notebook :

**L’application utilisée par le banquier nous envoie les informations suivantes :** 

_{
    "RevolvingUtilizationOfUnsecuredLines": 0.01703559,
    "NumberOfDependents": 1,
    "DebtRatio": 0,
    "age": 42,
    "NumberOfOpenCreditLinesAndLoans": 6,
    "NumberRealEstateLoansOrLines": 1,
    "NumberOfTime30-59DaysPastDueNotWorse": 1,
    "NumberOfTime60-89DaysPastDueNotWorse": 0,
    "NumberOfTimes90DaysLate": 0
}_

- Créez un nouveau notebook, dans lequel vous calculerez une probabilité de défaut de remboursement.
- Committez, et envoyez moi le lien pour que je retrouve facilement
- Dites “exo fini” ici-même, quand vous avez fini.

In [42]:
from bigml.api import BigML
from bigml.ensemble import Ensemble
from pandas import read_csv

# api = BigML(project='project/5d94a454eba31d46690001d1')
api = BigML()

df = read_csv('csv_file/exo_1input.csv', index_col=0)
# ce csv a été créé avec toutes les infos données par le banquier sur nouvel input

In [43]:
def change_dataset(dataframe):
    dataframe['IncomePerPerson'] = dataframe['MonthlyIncome'] / ( dataframe['NumberOfDependents'] + 1 )
    dataframe.loc[dataframe.age > 80, 'isOld'] = '1' 
    dataframe.loc[dataframe.age <= 80, 'isOld'] = '0'
    dataframe['MonthlyDebt'] = dataframe['MonthlyIncome'] * dataframe['DebtRatio']
    dataframe['MonthlyBalance'] = dataframe['MonthlyIncome'] - dataframe['MonthlyDebt']
    dataframe['DebtPerPerson'] = dataframe['MonthlyDebt'] / ( dataframe['NumberOfDependents'] + 1 )
    dataframe['BalancePerPerson'] = dataframe['MonthlyBalance'] / ( dataframe['NumberOfDependents'] + 1 )
    dataframe['NumberOfTime30-89DaysPastDueNotWorse'] = dataframe['NumberOfTime30-59DaysPastDueNotWorse'] + dataframe['NumberOfTime60-89DaysPastDueNotWorse']
    dataframe['NumbersOfOpen-NumberRealEstate'] = dataframe['NumberOfOpenCreditLinesAndLoans'] - dataframe['NumberRealEstateLoansOrLines']
    dataframe = dataframe.fillna(0)
    dataframe.index.names = ['Id']
    return dataframe

In [44]:
df_modify = change_dataset(df)

In [45]:
df_modify.to_csv('csv_file/input_prepared.csv')

In [47]:
model_ensemble = Ensemble('ensemble/5dc2d7d3e476847468013866', api=BigML(domain="bigml.io"))

TypeError: __init__() got an unexpected keyword argument 'domain'

In [36]:
input_source = api.create_source('csv_file/input_prepared.csv')
api.ok(input_source)

input_dataset = api.create_dataset(input_source)
api.ok(input_dataset)

True

In [39]:
input_prediction = model_ensemble.predict(input_dataset, full=True)
input_prediction

{'probability': 0.93283, 'prediction': '0'}